### Import libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model


### LOAD DATA

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)  # use 20% of data for validation

train_generator = train_datagen.flow_from_directory(
    r'C:\Users\gusta\Documents\RoadSign_Proj\RoadSignApp\pictures', # Data Directory PATH. CHANGE IF NEEDED
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    r'C:\Users\gusta\Documents\RoadSign_Proj\RoadSignApp\pictures', # Data Directory PATH. CHANGE IF NEEDED
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')


Found 419 images belonging to 419 classes.
Found 0 images belonging to 419 classes.


### LOAD PRETRAINED MODEL

In [4]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Freeze the base model
base_model.trainable = False




9406464/9406464 [==============================] - 1s 0us/step


### ADD LAYER

In [7]:
number_of_classes = 419 # CHANGE IF NEEDED

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(number_of_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


### COMPILE MODEL

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### TRAIN MODEL

In [10]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size,
    epochs=20)


Epoch 1/20
13/13 [==============================] - 9s 675ms/step - loss: 1.2095 - accuracy: 0.6770
Epoch 2/20
13/13 [==============================] - 9s 664ms/step - loss: 1.0854 - accuracy: 0.7313
Epoch 3/20
13/13 [==============================] - 9s 670ms/step - loss: 0.9714 - accuracy: 0.7416
Epoch 4/20
13/13 [==============================] - 9s 662ms/step - loss: 1.0555 - accuracy: 0.7054
Epoch 5/20
13/13 [==============================] - 9s 670ms/step - loss: 0.9129 - accuracy: 0.7700
Epoch 6/20
13/13 [==============================] - 9s 723ms/step - loss: 0.9678 - accuracy: 0.7158
Epoch 7/20
13/13 [==============================] - 9s 672ms/step - loss: 0.7383 - accuracy: 0.7829
Epoch 8/20
13/13 [==============================] - 9s 710ms/step - loss: 0.7489 - accuracy: 0.8243
Epoch 9/20
13/13 [==============================] - 10s 753ms/step - loss: 0.7147 - accuracy: 0.8475
Epoch 10/20
13/13 [==============================] - 9s 712ms/step - loss: 0.7251 - accuracy: 0.795

### SAVE MODEL

In [11]:
model.save('road_sign_model.h5')

c:\Users\gusta\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
